In [2]:
import pandas as pd
import numpy as np
import ast

# Creando los dataframes para alimentar mis APIs

Debido a que cada dataframe es tan extenso, para eficientizar costos de deployment, elegí para alimentar mis APIs prepocesar los datos y preparar un CSV especifico para cada uno, de esta manera evito que en cada consulta se gasten recursos por demas.

Tambien de esta manera reduje bastante el tamaño de cada DataFrame, pasando de mas de 600MB a solo 2MB en el mas extremo de los casos



In [3]:
df_games = pd.read_csv("data\df_games_clean.csv")
df_games.head(1)


,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,atributos
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140.0,Kotoshiro,"['Casual', 'Strategy', 'Simulation', 'Indie', ..."


In [4]:
df_items = pd.read_csv("data\df_items_clean.csv")
df_items.head(1)

,Unnamed: 0,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10.0,Counter-Strike,6.0,0.0


In [5]:
df_reviews = pd.read_csv("data\df_reviews_clean.csv")
df_reviews.head(1)

,user_id,user_url,posted,item_id,recommend,sentimiento
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250.0,True,2


## 1° API: userdata

+ **userdata( *`User_id` : str* )**:
    Debe devolver `cantidad` de dinero gastado por el usuario, el `porcentaje` de recomendación en base a reviews.recommend y `cantidad de items`.

In [6]:
#creamos un dataframe con las columnas que necesito

user_data_id_price = pd.DataFrame({"item_id": df_games["id"], "price": df_games["price"]})

In [7]:
user_data_id_price

,item_id,price
0,761140.0,4.99
1,643980.0,0.00
2,670290.0,0.00
3,767400.0,0.99
4,773570.0,2.99
...,...,...
31819,773640.0,1.99
31820,733530.0,4.99
31821,610660.0,1.99
31822,658870.0,4.99


In [8]:
#utilizo un dataframe auxiliar en donde joineo
df_aux = df_items.join(user_data_id_price.set_index('item_id'), on='item_id')

In [9]:
df_aux.head()

,Unnamed: 0,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,price
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10.0,Counter-Strike,6.0,0.0,9.99
1,1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20.0,Team Fortress Classic,0.0,0.0,4.99
2,2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30.0,Day of Defeat,7.0,0.0,4.99
3,3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40.0,Deathmatch Classic,0.0,0.0,4.99
4,4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50.0,Half-Life: Opposing Force,0.0,0.0,4.99


In [10]:
#elimino las columnas que no necesito que estaban en el dataframe df_items

user_data_id_price = df_aux.drop(columns= ['steam_id', 'user_url', 'playtime_forever', 'playtime_2weeks', 'Unnamed: 0', 'item_name','item_id'])

In [11]:
#agrupo por user_id

user_data = user_data_id_price.groupby('user_id').agg({'items_count': 'first', 'price': 'sum'}).reset_index()

In [12]:
#testeando
user_data

,user_id,items_count,price
0,--000--,58,402.77
1,--ace--,44,166.82
2,--ionex--,23,109.92
3,-2SV-vuLB-Kg,68,437.49
4,-404PageNotFound-,149,1514.31
...,...,...,...
70907,zzonci,5,19.98
70908,zzoptimuszz,61,64.98
70909,zzydrax,13,99.94
70910,zzyfo,84,828.51


In [13]:
#creo otro dataframe auxiliar ocn las columnas que necesito, user_id para joinear despues, recommend para sacar los datos.
df_aux2 = pd.DataFrame({"user_id": df_reviews["user_id"], "recommend": df_reviews["recommend"]})


In [14]:
#analizo porcentaje
df_aux2 = df_aux2.groupby('user_id')['recommend'].mean() * 100



In [15]:
df_aux2

user_id
--000--         100.0
--ace--         100.0
--ionex--       100.0
-2SV-vuLB-Kg    100.0
-Azsael-        100.0
                ...  
zwanzigdrei     100.0
zy0705          100.0
zynxgameth      100.0
zyr0n1c         100.0
zzoptimuszz     100.0
Name: recommend, Length: 25458, dtype: float64

In [16]:
df_aux2.to_frame()

,recommend
user_id,
--000--,100.0
--ace--,100.0
--ionex--,100.0
-2SV-vuLB-Kg,100.0
-Azsael-,100.0
...,...
zwanzigdrei,100.0
zy0705,100.0
zynxgameth,100.0


In [17]:
df_aux2.reset_index()

,user_id,recommend
0,--000--,100.0
1,--ace--,100.0
2,--ionex--,100.0
3,-2SV-vuLB-Kg,100.0
4,-Azsael-,100.0
...,...,...
25453,zwanzigdrei,100.0
25454,zy0705,100.0
25455,zynxgameth,100.0
25456,zyr0n1c,100.0


In [18]:
#joineo user_data con el dataframe auxiliar
user_data = user_data.join(df_aux2, on='user_id')

In [19]:
user_data["recommend"] = user_data["recommend"].fillna("Sin Recomendaciones")

In [20]:
#todo anda bien. tengo en un solo datadframe el total de los datos que necesito para las consultas de la API. 
user_data

,user_id,items_count,price,recommend
0,--000--,58,402.77,100.0
1,--ace--,44,166.82,100.0
2,--ionex--,23,109.92,100.0
3,-2SV-vuLB-Kg,68,437.49,100.0
4,-404PageNotFound-,149,1514.31,Sin Recomendaciones
...,...,...,...,...
70907,zzonci,5,19.98,Sin Recomendaciones
70908,zzoptimuszz,61,64.98,100.0
70909,zzydrax,13,99.94,Sin Recomendaciones
70910,zzyfo,84,828.51,Sin Recomendaciones


In [21]:
#user_data.to_csv('user_data.csv')

## 2° API: countreviews

**countreviews( *`YYYY-MM-DD` y `YYYY-MM-DD` : str* )**:
    `Cantidad de usuarios` que realizaron reviews entre las fechas dadas y, el `porcentaje` de recomendación de los mismos en base a reviews.recommend.


In [22]:
#creo un dataframe con las columnas que necesito del df_reviews
countreviews = pd.DataFrame({"review": df_reviews["user_id"], "posted": df_reviews["posted"], "recommend": df_reviews["recommend"]})

In [23]:
#review 1 es por que no me interesa el usuario especifico, si no que me interesa sumarlo
countreviews['review'] = 1

In [24]:
#agrupo por fecha, recomend va a ser por 

countreviews = countreviews.groupby('posted').agg(
    recommend_percentage=('recommend', lambda x: (x.sum() / len(x)) * 100),
    sum_review=('review', 'sum')
).reset_index()

In [25]:
#2015-12-31 / 2010-10-16  rari pero bueno.
#listo mi dataframe para responder consultas
countreviews

,posted,recommend_percentage,sum_review
0,2010-10-16,100.000000,1
1,2010-10-25,100.000000,1
2,2010-11-19,100.000000,1
3,2010-11-20,100.000000,2
4,2010-11-22,100.000000,3
...,...,...,...
1639,2015-12-27,88.235294,85
1640,2015-12-28,89.247312,93
1641,2015-12-29,87.012987,77
1642,2015-12-30,84.000000,75


In [26]:
countreviews["posted"] = pd.to_datetime(countreviews["posted"])

In [27]:
countreviews.dtypes

posted                  datetime64[ns]
recommend_percentage           float64
sum_review                       int64
dtype: object

In [28]:
#countreviews.to_csv("count_reviews.csv")

## 3° API: genre

**genre( *`género` : str* )**:
    Devuelve el `puesto` en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever. 

In [29]:
#primero creo los dataframes que voy a utilizar para hacer el dataframe final para la consulta
df_aux1 = pd.DataFrame({"genres" : df_games["genres"], "item_id" : df_games["id"]})
df_aux2 = pd.DataFrame({"item_id": df_items["item_id"], "playtime_forever": df_items["playtime_forever"]})

In [30]:
#joineo
genre = df_aux2.join(df_aux1.set_index('item_id'), on= 'item_id')

In [31]:
#borro los Nan por que me estaban dando problemas con la libreria ast.
genre = genre.dropna(subset=['genres'])

genre['genres'] = genre['genres'].apply(ast.literal_eval)

In [32]:
#cambio el tipo de datos para que pese menos y no haya posibilidad de error
genre["playtime_forever"] = genre["playtime_forever"].astype('int32')

In [33]:
#desanido la columna de genre, debido a que los juegos pueden ser que tengas mas de un genero. Tambien elimino la columna "item_id"
genre = genre.explode('genres')
genre.drop(columns=['item_id'])

,playtime_forever,genres
0,6,Action
1,0,Action
2,7,Action
3,0,Action
4,0,Action
...,...,...
5153206,3,Adventure
5153206,3,Free to Play
5153207,4,Casual
5153207,4,Free to Play


In [34]:
#agrupo por genero y sumo los playtimes
genre = genre.groupby('genres')['playtime_forever'].sum()

In [35]:
#testeando
genre.to_frame()
genre.reset_index()
genre = pd.DataFrame(genre)


In [36]:
genre = genre.sort_values(by="playtime_forever", ascending=False)

In [37]:
top = []
for i in range(1,22):
    top.append(f"Top {i}")
    

In [38]:
genre["Top"] = top

In [39]:
#genre.to_csv("genre.csv")

## 4° API: userforgenre

**userforgenre( *`género` : str* )**:
    `Top 5` de usuarios con más horas de juego en el género dado, con su URL y user_id.

In [40]:
#creo dataframes auxiliares con las columnas que voy a utilizar
df_aux1 = pd.DataFrame({"genres" : df_games["genres"], "item_id" : df_games["id"]})
df_aux2 = pd.DataFrame({"item_id": df_items["item_id"], "playtime_forever": df_items["playtime_forever"], 'user_url' : df_items['user_url'], 'user_id' : df_items["user_id"]})

In [41]:
df_aux1 = df_aux1.dropna(subset= 'genres')
df_aux1['genres'] = df_aux1['genres'].apply(ast.literal_eval)
df_aux1 = df_aux1.explode("genres")

In [42]:
df_aux1

,genres,item_id
0,Action,761140.0
0,Casual,761140.0
0,Indie,761140.0
0,Simulation,761140.0
0,Strategy,761140.0
...,...,...
31821,Racing,610660.0
31821,Simulation,610660.0
31822,Casual,658870.0
31822,Indie,658870.0


In [43]:
userforgenre = df_aux2.join(df_aux1.set_index('item_id'), on= 'item_id')
userforgenre["playtime_forever"] = userforgenre["playtime_forever"].astype('int32')
userforgenre = userforgenre.drop(columns= 'item_id')

In [44]:
#despues de joinear asi es como queda, podria dejarlo así pero se puede procesar mas aún
userforgenre

,playtime_forever,user_url,user_id,genres
0,6,http://steamcommunity.com/profiles/76561197970...,76561197970982479,Action
1,0,http://steamcommunity.com/profiles/76561197970...,76561197970982479,Action
2,7,http://steamcommunity.com/profiles/76561197970...,76561197970982479,Action
3,0,http://steamcommunity.com/profiles/76561197970...,76561197970982479,Action
4,0,http://steamcommunity.com/profiles/76561197970...,76561197970982479,Action
...,...,...,...,...
5153206,3,http://steamcommunity.com/profiles/76561198329...,76561198329548331,Free to Play
5153207,4,http://steamcommunity.com/profiles/76561198329...,76561198329548331,Casual
5153207,4,http://steamcommunity.com/profiles/76561198329...,76561198329548331,Free to Play
5153207,4,http://steamcommunity.com/profiles/76561198329...,76561198329548331,Indie


In [45]:
userforgenre = userforgenre.groupby(['user_url', 'user_id', 'genres'], as_index=False)['playtime_forever'].sum()

Es posbile procesarla mas aún a este dataframe dadas las caracteristicas de la peticion. Unicamente tengo que almacenar 5 registros por genero, lo cual reduciría considerablemente el consumo de procesamiento y el tamaño del csv, sin embargo los tiempos de espera son  mas que aceptables y la cantidad de procesamiento tambien. Por ahora no lo veo necesario

In [46]:
userforgenre.head(20)

,user_url,user_id,genres,playtime_forever
0,http://steamcommunity.com/id/--000--,--000--,Action,139469
1,http://steamcommunity.com/id/--000--,--000--,Adventure,11722
2,http://steamcommunity.com/id/--000--,--000--,Casual,16135
3,http://steamcommunity.com/id/--000--,--000--,Early Access,531
4,http://steamcommunity.com/id/--000--,--000--,Free to Play,20448
5,http://steamcommunity.com/id/--000--,--000--,Indie,12218
6,http://steamcommunity.com/id/--000--,--000--,Massively Multiplayer,12788
7,http://steamcommunity.com/id/--000--,--000--,RPG,7362
8,http://steamcommunity.com/id/--000--,--000--,Simulation,1248
9,http://steamcommunity.com/id/--000--,--000--,Sports,5963


In [47]:
#userforgenre.to_csv("userforgenre.csv")

## 5° API: developer

**developer( *`desarrollador` : str* )**:
    `Cantidad` de items y `porcentaje` de contenido Free por año según empresa desarrolladora. 


In [48]:
#creo mi df nuevo con las columnas que voy a necesitar
developer = pd.DataFrame({"developer": df_games["developer"],"release_date": df_games["release_date"], "price": df_games["price"]})

In [49]:
#reemplazamos los "NoData" por NaN asi puedo utilizar funciones para cambiar a datetime
developer["release_date"] = developer["release_date"].replace("NoData", np.nan) 

In [50]:
#reemplazo todos las fecha por su año unicamente
developer['release_date'] = pd.to_datetime(developer['release_date'])
developer['release_date'] = developer['release_date'].dt.year

In [51]:
developer = developer.groupby(['developer', 'release_date'])['price'].apply(lambda x: (x == 0).mean()).reset_index()


In [52]:
#testeando
developer.query("developer == 'Gogii Games'")


,developer,release_date,price
5326,Gogii Games,2007.0,0.0
5327,Gogii Games,2008.0,0.0
5328,Gogii Games,2010.0,0.0
5329,Gogii Games,2012.0,0.0
5330,Gogii Games,2014.0,0.0


In [53]:
developer["release_date"] = developer["release_date"].astype("int16")
developer.rename(columns={"price": 'porcentaje'})
#developer.to_csv("developer.csv")

,developer,release_date,porcentaje
0,+7 Software,2016,0.0
1,"+Mpact Games, LLC.",2017,0.0
2,.M.Y.W.,2016,0.0
3,.ez Games,2017,0.0
4,07th Expansion,2015,0.0
...,...,...,...
14850,萌石游戏,2017,0.0
14851,高考恋爱委员会,2015,1.0
14852,"高考恋爱委员会,Days",2015,0.0
14853,"高考恋爱委员会,橘子班",2015,0.0


## 6° API: sentiment_analysis

**sentiment_analysis( *`empresa desarrolladora` : str* )**:
    Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento. 




<br/>

In [54]:
#creamos el primer df que vamos a usar
df_aux = pd.DataFrame({"release_date": df_games["release_date"],"item_id" : df_games["id"]})

In [55]:
#reemplazamos los "NoData" por NaN asi puedo utilizar funciones para cambiar a datetime
df_aux["release_date"] = df_aux["release_date"].replace("NoData", np.nan) 
df_aux['release_date'] = pd.to_datetime(df_aux['release_date'])
df_aux['release_date'] = df_aux['release_date'].dt.year

In [56]:
df_aux 

,release_date,item_id
0,2018.0,761140.0
1,2018.0,643980.0
2,2017.0,670290.0
3,2017.0,767400.0
4,NaN,773570.0
...,...,...
31819,2018.0,773640.0
31820,2018.0,733530.0
31821,2018.0,610660.0
31822,2017.0,658870.0


In [57]:
#creamos el segundo df que vamos a usar
df_aux2 = pd.DataFrame({"sentimiento" : df_reviews["sentimiento"], "item_id" : df_reviews["item_id"]})

In [58]:
df_aux2

,sentimiento,item_id
0,2,1250.0
1,2,22200.0
2,2,43110.0
3,2,251610.0
4,2,227300.0
...,...,...
59300,2,70.0
59301,2,362890.0
59302,2,273110.0
59303,2,730.0


In [59]:
#hacemos la agrupacion por item_id
df_aux2 = df_aux2.groupby('item_id')['sentimiento'].value_counts().unstack(fill_value=0)

In [60]:
#eureka
df_aux2

sentimiento,0,1,2
item_id,,,
10.0,1,23,33
20.0,2,5,10
30.0,0,1,3
40.0,0,0,1
50.0,0,1,3
...,...,...,...
521340.0,0,0,2
521430.0,0,0,1
521570.0,2,0,0


In [61]:
#hago un join por "item_id"
sentiment_analysis = df_aux2.join(df_aux.set_index('item_id'), on='item_id')

In [62]:
sentiment_analysis

,0,1,2,release_date
item_id,,,,
10.0,1,23,33,2000.0
20.0,2,5,10,1999.0
30.0,0,1,3,2003.0
40.0,0,0,1,2001.0
50.0,0,1,3,1999.0
...,...,...,...,...
521340.0,0,0,2,2016.0
521430.0,0,0,1,2016.0
521570.0,2,0,0,2016.0


In [63]:
sentiment_analysis.set_index('release_date', inplace=True)  # Establecer 'release_date' como índice
sentiment_analysis = sentiment_analysis.groupby('release_date').sum() #agrupo por año

In [64]:
sentiment_analysis

,0,1,2
release_date,,,
1989.0,0,0,1
1990.0,1,0,4
1991.0,0,0,1
1992.0,1,0,2
1993.0,1,1,3
1994.0,3,4,8
1995.0,5,1,10
1996.0,0,3,5
1997.0,8,7,33


In [65]:
#sentiment_analysis.to_csv('sentiment_analysis.csv')


## 7° API: recomend (modelo de ML)

**recomend( *`id_del_producto` : int* )**:
    Recomienda un juego en base al id del juego brindado. 
<br/>

#### Analisis exploratorio para para modelado.

Para crear mi modelo de machine learning primero voy a realizar un pequeño EDA orientado para comprender el contenido de mis datos y asegurar que no hayan interferencias futuras. Este modelo se va a bazar en en unicamente 2 dataframes:

- df_reviews
- df_games  


#### Data Frame REVIEWS

In [ ]:
#Vemos el tamaño de nuestro DF
df_reviews.shape

(59305, 6)

In [ ]:
#hacemos un sampleo aleatorio para ver como esta estructurada nuestra informacion
df_reviews.sample(5)

,user_id,user_url,posted,item_id,recommend,sentimiento
8779,76561198009000573,http://steamcommunity.com/profiles/76561198009...,NaN,381210.0,False,0
38041,76561198073259913,http://steamcommunity.com/profiles/76561198073...,2015-07-12,274940.0,True,1
44790,jujubamagica,http://steamcommunity.com/id/jujubamagica,2014-08-18,550.0,True,1
2104,76561198026040814,http://steamcommunity.com/profiles/76561198026...,2015-11-26,211420.0,True,0
1499,Clouwolf,http://steamcommunity.com/id/Clouwolf,NaN,730.0,False,2


In [ ]:
#nombres de las columnas
df_reviews.columns

Index(['user_id', 'user_url', 'posted', 'item_id', 'recommend', 'sentimiento'], dtype='object')

In [ ]:
#posted deveria ser un datetime, no un object
df_reviews.dtypes

user_id         object
user_url        object
posted          object
item_id        float64
recommend         bool
sentimiento      int64
dtype: object

### Preparacion de Datos


In [ ]:
df_reviews["posted"] = pd.to_datetime(df_reviews["posted"])

In [ ]:
df_reviews.dtypes

user_id                object
user_url               object
posted         datetime64[ns]
item_id               float64
recommend                bool
sentimiento             int64
dtype: object

In [ ]:
#encontramos algo similar con respecto al df anterior, user_id y user_url son referenciales a lo mismo. Por ahora los vamos a dejar.
df_reviews.sample(3)

,user_id,user_url,posted,item_id,recommend,sentimiento
21122,76561198018469813,http://steamcommunity.com/profiles/76561198018...,2014-01-26,440.0,True,2
870,Aurora99,http://steamcommunity.com/id/Aurora99,2014-09-15,72850.0,True,1
31389,ultimatefaggotwow,http://steamcommunity.com/id/ultimatefaggotwow,2014-01-31,204300.0,True,1


In [ ]:
#intentando identificar si aun tenemos valores faltantes. 
df_reviews.isna().sum()

user_id            0
user_url           0
posted         10119
item_id            0
recommend          0
sentimiento        0
dtype: int64

Los nulos constituyen casi un 20% del total de los datos de la columna de fecha, lo cual dificulta un poco el analisis. 
En este caso voy a eliminar la columna ya que no la necesito para hacer mi modelado y no me va a aportar un valor significativo. 
Si en vez de eliminar la columna, elimino las filas que contienen nulos, me quedo con los datos de la fecha que fuero posteados pero pierdo mucha informacion valiosa sobre todo en la columna recommend y sentimiento

In [ ]:
df_reviews.drop(columns=['posted'], inplace= True)

In [ ]:
df_reviews.sample(3)

,user_id,user_url,item_id,recommend,sentimiento
39820,geekosaurus,http://steamcommunity.com/id/geekosaurus,363970.0,True,0
42352,ToxicK-9,http://steamcommunity.com/id/ToxicK-9,550.0,True,0
47906,76561198045627858,http://steamcommunity.com/profiles/76561198045...,372000.0,False,1


## Data Frame GAMES

In [ ]:
#Vemos el tamaño de nuestro DF
df_games.shape

(31824, 12)

In [ ]:
#hacemos un sampleo aleatorio para ver como esta estructurada nuestra informacion
df_games.sample(5)

,genres,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,atributos
3521,"['Action', 'Adventure', 'Indie', 'Racing']",JumpJet Rex - Soundtrack,http://store.steampowered.com/app/364400/JumpJ...,2015-04-21,"['Action', 'Adventure', 'Indie', 'Racing']",http://steamcommunity.com/app/364400/reviews/?...,"['Single-player', 'Shared/Split Screen', 'Down...",3.99,0.0,364400.0,TreeFortress Games,"['Racing', 'Adventure', 'Stats', 'Full control..."
3658,"['Action', 'Indie', 'Simulation']",Luna's Wandering Stars,http://store.steampowered.com/app/330660/Lunas...,2015-05-15,"['Simulation', 'Action', 'Indie', 'Puzzle', 'L...",http://steamcommunity.com/app/330660/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",9.99,0.0,330660.0,Serenity Forge,"['Space', 'Level Editor', 'Great Soundtrack', ..."
10331,"['Adventure', 'Casual', 'Simulation']",Libra of the Vampire Princess: Lycoris &amp; A...,http://store.steampowered.com/app/644680/Libra...,2017-07-07,"['Adventure', 'Casual', 'Simulation', 'Visual ...",http://steamcommunity.com/app/644680/reviews/?...,['Single-player'],15.99,0.0,644680.0,"onomatope*,MiKandi Japan","['Casual', 'Anime', 'Adventure', 'Simulation',..."
29289,"['Adventure', 'RPG', 'Simulation']",Starlite: Astronaut Rescue - Developed in Coll...,http://store.steampowered.com/app/266090/Starl...,2014-01-27,"['Adventure', 'RPG', 'Simulation', 'Space', 'S...",http://steamcommunity.com/app/266090/reviews/?...,['Single-player'],2.99,0.0,266090.0,Project Whitecard Studios Inc.,"['Space', 'Adventure', 'Short', 'Simulation', ..."
19689,"['Adventure', 'Indie']",Behind the Memory,http://store.steampowered.com/app/568340/Behin...,2016-11-25,"['Adventure', 'Indie', 'Walking Simulator']",http://steamcommunity.com/app/568340/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",3.99,0.0,568340.0,GuraShop,"['Partial Controller Support', 'Walking Simula..."


In [ ]:
#id refiere a item_id.
#tambien tenemos la columna que cree antes de atributos con la union de genres, specs y tags.

df_games.columns

Index(['genres', 'title', 'url', 'release_date', 'tags', 'reviews_url',
       'specs', 'price', 'early_access', 'id', 'developer', 'atributos'],
      dtype='object')

#### Preparacion de Datos

In [ ]:
#voy a eliminar las columnas de specs, genres y tags para hacer el modelado ya que para mi modelado solo voy a usar mi columna atributos

df_games.rename(columns= {'id' : "item_id"}, inplace=True)
df_games.drop(columns=["genres","tags","specs"], inplace=True)

In [ ]:
#Hay que acomodar algunos tipos de datos como "early_access", "release_date" y transformar a dummies "atributos"
df_games.dtypes

title            object
url              object
release_date     object
reviews_url      object
price           float64
early_access    float64
item_id         float64
developer        object
atributos        object
dtype: object

In [ ]:
df_games['release_date'] = df_games['release_date'].replace("NoData", np.nan)
df_games['release_date'] = pd.to_datetime(df_games['release_date'])
df_games["early_access"] = df_games["early_access"].astype(bool)
df_games['atributos'] = df_games['atributos'].apply(ast.literal_eval)


In [ ]:
df_games.dtypes

title                   object
url                     object
release_date    datetime64[ns]
reviews_url             object
price                  float64
early_access              bool
item_id                float64
developer               object
atributos               object
dtype: object

In [ ]:
#aislo las columnas mas reelevantes de cada DF. Decido hacer un modelo de recomendacion basado en los tags de los juegos por lo que no voy a utilizar ahora el df_items
df_games.drop(columns=["url","release_date","reviews_url","early_access"],inplace=True)
df_reviews.drop(columns=["user_url","user_id",], inplace=True)

In [ ]:
#juegos con mas recomendaciones
df_reviews = df_reviews.replace({True: 1, False:-1}).groupby('item_id').agg({'recommend': 'sum', 'sentimiento': 'mean'}).reset_index()

In [ ]:
df_reviews

,item_id,recommend,sentimiento
0,10.0,55,1.561404
1,20.0,5,1.470588
2,30.0,2,1.750000
3,40.0,1,2.000000
4,50.0,2,1.750000
...,...,...,...
3677,521340.0,2,2.000000
3678,521430.0,1,2.000000
3679,521570.0,2,0.000000
3680,521990.0,1,2.000000


In [ ]:
#voy a crear un indice entre recommend y sentimineto.
df_reviews['score'] = df_reviews['recommend'] / df_reviews['recommend'].max() + df_reviews['sentimiento'] / df_reviews['sentimiento'].max()
df_reviews['score'] = (df_reviews['score'] - df_reviews['score'].min()) / (df_reviews['score'].max() - df_reviews['score'].min())
df_reviews.drop(columns=["recommend", "sentimiento"], inplace=True)

In [ ]:
df_reviews

,item_id,score
0,10.0,0.446625
1,20.0,0.413156
2,30.0,0.490929
3,40.0,0.560787
4,50.0,0.490929
...,...,...
3677,521340.0,0.560947
3678,521430.0,0.560787
3679,521570.0,0.000803
3680,521990.0,0.560787


In [ ]:
#elimino espacios de cada uno de los atributos y las columnas que ya no necesito
df_games["atributos"] = df_games["atributos"].apply(lambda x:[i.replace(" ","") for i in x])
df_games["atributos"] = df_games["atributos"].apply(lambda x: ', '.join(map(str, x)))
df_games["tags"] = df_games["atributos"]+df_games["price"].astype(str)+df_games["developer"]
df_games.drop(columns=["atributos","developer","price"], inplace=True)
df_games["tags"] = df_games["tags"].apply(lambda x: x.lower())


In [ ]:
df_games

,title,item_id,tags
0,Lost Summoner Kitty,761140.0,"casual, strategy, simulation, indie, single-pl..."
1,Ironbound,643980.0,"pvp, replayvalue, charactercustomization, fant..."
2,Real Pool 3D - Poolians,670290.0,"casual, stats, simulation, indie, single-playe..."
3,弹炸人2222,767400.0,"adventure, action, single-player, casual0.99彼岸领域"
4,Log Challenge,773570.0,"casual, trackedmotioncontrollers, htcvive, ful..."
...,...,...,...
31819,Colony On Mars,773640.0,"casual, strategy, simulation, indie, single-pl..."
31820,LOGistICAL: South Africa,733530.0,"casual, stats, strategy, indie, single-player,..."
31821,Russian Roads,610660.0,"racing, steamtradingcards, simulation, indie, ..."
31822,EXIT 2 - Directions,658870.0,"casual, indie, single-player, steamachievement..."


In [ ]:
df_itemsxitems = pd.merge(df_games, df_reviews, on="item_id", how="outer")
df_itemsxitems = df_itemsxitems.dropna(subset="title").fillna(0).sort_values(by="score", ascending=False)

In [ ]:
df_itemsxitems

,title,item_id,tags,score
31577,Team Fortress 2,440.0,"robots, class-based, stats, fps, first-person,...",1.000000
1034,Counter-Strike: Global Offensive,730.0,"pvp, stats, fps, realistic, difficult, first-p...",0.921146
53,Garry's Mod,4000.0,"animation&modeling, fps, steamcloud, first-per...",0.698078
1451,Dota 2,570.0,"pvp, replayvalue, charactercustomization, htcv...",0.630267
30979,Terraria,105600.0,"replayvalue, onlineco-op, onlinemulti-player, ...",0.564298
...,...,...,...,...
11458,Toradora!,694620.0,"episodic, anime0.99nodata",0.000000
11457,Celtabula,706200.0,"casual, puzzle, single-player, indie1.99skip32",0.000000
11456,TANE DLC: Orient Express Trainset,715080.0,"simulation, co-op, single-player, multi-player...",0.000000
11455,PetriDish.pw,688730.0,"casual, partialcontrollersupport, massivelymul...",0.000000


In [ ]:
#exporto mis datos en un archivo csv.
df_itemsxitems.to_csv("datos_ML.csv")
df_itemsxitems.head(2000).to_csv("datos_ML_reduced.csv")